Importando bibliotecas

In [1]:
import pandas as pd
import json
import gensim
import nltk
from  nltk.corpus import stopwords
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import common_texts, get_tmpfile
import datetime
import numpy as np
from sklearn.manifold import TSNE
from bokeh.models import ColumnDataSource, LabelSet, LayoutDOM, Plot
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
from bokeh.core.properties import Instance, String
from bokeh.resources import INLINE
import bokeh.io
from scipy.spatial import distance

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Importando base concatenada e misturando os artigos para evitar que haja um viés devido a ordem de leitura dos documentos.

In [3]:
abstract_OG = pd.read_json("Elsevier_abstract - Consolidado.json")
# Misturando as linhas e resetando o index
abstract_OG = abstract_OG.sample(frac=1).reset_index(drop=True)

In [4]:
# Contando os documentos
print("Núlen(abstract_OG)

35342

## Pré-processamento

Retirando palavras e espaços

In [428]:
# Retirando as palavras "Abstract", "Unknown", "Publisher Summary", "Summary", "Fundamento"
# que aparece no início de vários textos
#abstract_OG["Abstract"] = abstract_OG["Abstract"].str.replace("Abstract", "")
#abstract_OG["Abstract"] = abstract_OG["Abstract"].str.replace("Unknown", "")
#abstract_OG["Abstract"] = abstract_OG["Abstract"].str.replace("Publisher Summary", "")
#abstract_OG["Abstract"] = abstract_OG["Abstract"].str.replace("Summary", "")
#abstract_OG["Abstract"] = abstract_OG["Abstract"].str.replace("Fundamento", "")

In [429]:
# Separando o texto em uma lista de palavras desconsiderando espaços e \n
#abstract_OG["Abstract"] = abstract_OG["Abstract"].str.split()      
# Unificando a lista de palavras usando espaços simples
#abstract_OG["Abstract"] = abstract_OG["Abstract"].str.join(" ")

Eliminar as linhas sem Abstract

In [430]:
#abstract_OG = abstract_OG[abstract_OG["Abstract"] != ""]
#abstract_OG = abstract_OG.reset_index()
#abstract_OG = abstract_OG.drop(columns=['index'])

Realizando o pré-processamento simple do Gensim basicamente para gerar os tokens dos textos

In [431]:
#abstract_OG["Abstract"] = abstract_OG["Abstract"].apply(gensim.utils.simple_preprocess)

In [432]:
# Importando as bibliotecas de stopwords
#nltk.download('stopwords')
# Mapeando stopwords com NLTK
#stopwordsIngles = stopwords.words("english")

#def remove_stopwords(abstract):
#    without_stopwords = []
#    for word in abstract:
#        if word not in stopwordsIngles:
#            without_stopwords.append(word)
#    return(without_stopwords)
    
# Excluindo stopwords
#abstract_OG["Abstract"] = abstract_OG["Abstract"].apply(remove_stopwords)

Fazer pré-processamento das keywords e Creators

In [433]:
#Procurar "keyword" e "creator" em todos os artigos de O&G

#momentoInicial = datetime.datetime.now()   # Inicia um contador do tempo

#n = 0

#for index, row in abstract_OG.iterrows():
#    n = n+1
#    preprocess_key = []
#    preprocess_creator = []
    
#    row_keywords = row["Keywords"]  # Selecionar keywords
#    for word in row_keywords:       # Looping em cada keyword
#        word = word.strip()         # Retirando espaços em branco
#        word = word.lower()         # Todas letras em minúsculo
#        preprocess_key.append(word)
        
#    row_creators = row["Creator"]   # Selecionar autores
#    for word in row_creators:       # Looping em cada keyword
#        word = word.strip()         # Retirando espaços em branco
#        word = word.lower()         # Todas letras em minúsculo
#        preprocess_creator.append(word)
        
#    abstract_OG["Keywords"][index] = preprocess_key
#    abstract_OG["Creator"][index] = preprocess_creator

#    if n%1000 == 0:
#        print ("Artigos pré-processados: ", n)
        
#momentoFinal = datetime.datetime.now() #Encerrando o contador do tempo
#print("Tempo total decorrido: ", momentoFinal - momentoInicial)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Artigos pré-processados:  1000
Artigos pré-processados:  2000
Artigos pré-processados:  3000
Artigos pré-processados:  4000
Artigos pré-processados:  5000
Artigos pré-processados:  6000
Artigos pré-processados:  7000
Artigos pré-processados:  8000
Artigos pré-processados:  9000
Artigos pré-processados:  10000
Artigos pré-processados:  11000
Artigos pré-processados:  12000
Artigos pré-processados:  13000
Artigos pré-processados:  14000
Tempo total decorrido:  4:59:56.925988


Gravando arquivo Pré-processado

In [327]:
#abstract_OG.to_json("Elsevier_abstract - Preprocessado.json", date_format = 'iso')
abstract_OG = pd.read_json("Elsevier_abstract - Preprocessado.json")

## Vetorizando os documentos

Gerando os modelos Doc2Vec utilizando os artigos provenientes do conjunto de treino

In [328]:
momentoInicial = datetime.datetime.now()   # Inicia um contador do tempo

# Todos os documentos devem passar pela função TaggedDocument antes de ser vetorizado
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(abstract_OG["Abstract"])]

#Hiperparâmetros do modelo Doc2Vec
Abstract_doc2vec = Doc2Vec(documents,
                           vector_size=8,
                           dm=1,
                           window=5,
                           min_count=250,
                           workers=7,
                           epochs=300,
                           alpha=0.025,
                           min_alpha=0.00025)

momentoFinal = datetime.datetime.now() #Encerrando o contador do tempo
print("Tempo total decorrido: ", momentoFinal - momentoInicial)

Tempo total decorrido:  0:09:10.297600


In [329]:
# Gravando modelo Doc2Vec
fname = "AbstractKG_doc2vec_model"
Abstract_doc2vec.save(fname)

In [330]:
# Lendo modelo Doc2Vec
fname = "AbstractKG_doc2vec_model"
Abstract_doc2vec = Doc2Vec.load(fname)

In [331]:
# Utilizar o modelo Doc2Vec gerado para inferir os vetores para todos os artigos.

momentoInicial = datetime.datetime.now()   # Inicia um contador do tempo

abstract_OG["Doc2Vec"] = abstract_OG["Abstract"].apply(Abstract_doc2vec.infer_vector)

momentoFinal = datetime.datetime.now() #Encerrando o contador do tempo
print("Tempo total decorrido: ", momentoFinal - momentoInicial)

Tempo total decorrido:  0:07:21.377400


Gravando arquivo com Doc2Vec inferido

In [332]:
abstract_OG.to_json("Elsevier_abstract - Preprocessado.json", date_format = 'iso')
abstract_OG = pd.read_json("Elsevier_abstract - Preprocessado.json")

In [333]:
# Retornando um exemplo com os 6 documentos mais similares (o próprio + top5)
top5 = Abstract_doc2vec.docvecs.most_similar([Abstract_doc2vec.docvecs[0]], topn=6)
for n in top5:
    print("Abstract ", n, "----> ", abstract_OG.loc[n[0]]["Title"])

Abstract  (0, 1.0) ---->  A Wide-Bandwidth Ultrasonic Study of Suspensions: The Variation of Velocity and Attenuation with Particle Size 
Abstract  (9970, 0.9701407551765442) ---->  Anaerobic biodegradability testing of surfactants 
Abstract  (14098, 0.9677221179008484) ---->  Fullerene grafted hydrocarbon polymers 
Abstract  (7425, 0.9658060669898987) ---->  The economic control of quality 
Abstract  (10188, 0.9613857865333557) ---->  Valence force calculation of the vibrational spectra of crystalline isotactic polypropylene and some deuterated polypropylenes 
Abstract  (1, 0.9543232917785645) ---->  Elongation and subsequent relaxation measurements on dilute polyisobutylene solutions 


C:\ProgramData\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


## Vetorizando as keywords

Nesta seção serão gerados vetores para as principais keywords que aparecem nos artigos de O&G. Para tal, será gerado uma lista de keywords. Para cada keyword dessa lista serão identificados e concatenados os abstracts a qual essa keyword está relacionada. O vetor da keyword será o vetor Doc2Vec inferido para o texto dos abstracts concatenados utilizado o modelo já treinado. 

In [334]:
# Identificando as keywords de O&G que parecem mais de 5 vezes

# frequência mínima para uma keyword ser considerada
#min_freq = 6 
# Separando os artigos de O&G
#keyword_list = abstract_OG
#keywords = []

# Incluindo keywords de cada artigo na lista "keywords"
#keyword_list["Keywords"].apply(keywords.extend)               

# Criando DataFrame com a frequência das keywords
#keywords = pd.Series(keywords)
#keywords = pd.DataFrame(data={"Frequency": keywords.value_counts()})
# Retirand as keywords com Frequência menor do que min_freq
#keywords = keywords[keywords["Frequency"] >= min_freq]

In [335]:
# Concatenando todos os abstracts de uma mesma keyword

#momentoInicial = datetime.datetime.now()   # Inicia um contador do tempo

# Criando DataFrame para armazenar os Abstracts concatenados das keywords
#keyword_vectors = pd.DataFrame(columns=['Keywords', 'Abstracts'])

# Fazer uma iteração para todas as keywords
#for key in keywords.index:
#    abstracts = []
      
     #Procurar a keyword em todos os artigos de O&G
#    for index, row in abstract_OG.iterrows():

        # Verificar se a Keyword procurada está nesse artigo
#        if key in row["Keywords"]:
            # Concatenar abstracts
#            abstracts.extend(row["Abstract"])
     
    # Incluir linha no DataFrame "keyword_vectors"
#    keyword_vectors = keyword_vectors.append({'Keywords': key, 'Abstracts': abstracts}, ignore_index=True)
    
#momentoFinal = datetime.datetime.now() #Encerrando o contador do tempo
#print("Tempo total decorrido: ", momentoFinal - momentoInicial)        

In [336]:
# Para não precisar repetir as etapas anteriores (separar keywords e concatenar os textos) lemos diretamento do disco
keyword_vectors = pd.read_json("Elsevier_abstract - Keywords.json")

In [337]:
# Utilizando o modelo já treinado inferir os vetores para as keywords
momentoInicial = datetime.datetime.now()   # Inicia um contador do tempo

keyword_vectors["Doc2Vec"] = keyword_vectors["Abstracts"].apply(Abstract_doc2vec.infer_vector)

momentoFinal = datetime.datetime.now() #Encerrando o contador do tempo
print("Tempo total decorrido: ", momentoFinal - momentoInicial)        

Tempo total decorrido:  0:06:56.420000


In [338]:
# Salvando e lendo em disco a lista de keyword com abstracts
keyword_vectors.to_json("Elsevier_abstract - Keywords.json", date_format = 'iso')
keyword_vectors = pd.read_json("Elsevier_abstract - Keywords.json")

In [339]:
keyword_vectors.head()

,Abstracts,Doc2Vec,Keywords
0,"[the, co, capture, project, ccp, phase, progra...","[3.0489015579, 4.9259805679, 0.7291272879, 0.8...",co2
1,"[this, study, deals, with, the, application, o...","[1.2594091892, 1.9156749249, -0.6412436366, -0...",modelling
10,"[process, evaluation, of, aucl, mg, ph, donor,...","[2.3217506409, 0.5866640806, 2.0094738007, 3.6...",kinetics
100,"[this, paper, documents, the, discovery, of, f...","[3.5142683983, 3.5081384182, 1.441160202, -4.2...",overpressure
101,"[although, the, term, naphthenic, acids, was, ...","[-2.4418046474, 1.4438048601, -1.8562635183, 0...",mass spectrometry


É necessário reduzir a dimensionalidade dos vetores de 50 para 2 para conseguir gerar uma visualização. Será utilizado o algoritmo tSNE.

In [340]:
# Reduzindo de 50 dimensões para 2 usando tSNE para permitir a plotagem
tsne = TSNE(n_components=2, random_state=0, perplexity=12,early_exaggeration=10  )
embed_tsne = tsne.fit_transform(keyword_vectors["Doc2Vec"].tolist())
embed_tsne = pd.DataFrame(data=embed_tsne)

In [2]:
#Definindo uma função que recebe os vetores já reduzidos para duas dimensões e plota o gráfico
def scater_plot(embed_tsne, vocabulario, color):
    p = figure(tools="pan,wheel_zoom,reset",#save",
               toolbar_location="above",
               title=" ",
               plot_width=800, 
               plot_height=500)

    source = ColumnDataSource(data=dict(x1=embed_tsne[0],
                                        x2=embed_tsne[1],
                                        names=vocabulario))

    p.scatter(x="x1", y="x2", size=8, source=source, color=color)

    labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                      text_font_size="8pt", text_color="#555555",
                      source=source, text_align='center')
    p.add_layout(labels)
    bokeh.io.output_notebook(INLINE) 
    output_notebook()
    show(p)
    return()

# Plotando o grafico
scater_plot(embed_tsne, keyword_vectors["Keywords"], 'CornflowerBlue')

## Vetorizando os autores

A mesma técnica utilizada para se criar os vetores das Keywords será utilizada para se vetorizar os autores dos documentos.

In [342]:
# Identificando autores de O&G que parecem mais de 5 vezes

# frequência mínima para um autor ser considerada
#min_freq = 6
# Separando os artigos de O&G
#creator_list = abstract_OG
#creators = []
# Incluindo autor de cada artigo na lista "creators"
#creator_list["Creator"].apply(creators.extend)               

# Criando DataFrame com a frequência dos autores
#creators = pd.Series(creators)
#creators = pd.DataFrame(data={"Frequency": creators.value_counts()})
# Retirand as keywords com Frequência menor do que min_freq
#creators = creators[creators["Frequency"] >= min_freq]

In [343]:
# Concatenando todos os abstracts de um mesmo autor

#momentoInicial = datetime.datetime.now()   # Inicia um contador do tempo

# Criando DataFrame para armazenar os Abstracts concatenados dos autores
#creator_vectors = pd.DataFrame(columns=['Creator', 'Abstracts'])

# Fazer uma iteração para todos os autores
#for key in creators.index:
    
#    abstracts = []
    
    #Procurar os autores em todos os artigos de O&G
#    for index, row in abstract_OG.iterrows():

        # Verificar se o autor procurado está nesse artigo
#        if key in row["Creator"]:
            # Concatenar abstracts
#            abstracts.extend(row["Abstract"])
     
    # Incluir linha no DataFrame "creator_vectors"
#    creator_vectors = creator_vectors.append({'Creator': key, 'Abstracts': abstracts}, ignore_index=True)
    
#momentoFinal = datetime.datetime.now() #Encerrando o contador do tempo
#print("Tempo total decorrido: ", momentoFinal - momentoInicial)        

In [4]:
# Para não precisar repetir as etapas anteriores (separar autores e concatenar os textos) lemos diretamento do disco
creator_vectors = pd.read_json("Elsevier_abstract - Creators.json")

In [5]:
len(creator_vectors)

941

In [345]:
# Utilizando o modelo já treinado inferir os vetores para os autores
momentoInicial = datetime.datetime.now()   # Inicia um contador do tempo

creator_vectors["Doc2Vec"] = creator_vectors["Abstracts"].apply(Abstract_doc2vec.infer_vector)

momentoFinal = datetime.datetime.now() #Encerrando o contador do tempo
print("Tempo total decorrido: ", momentoFinal - momentoInicial)

Tempo total decorrido:  0:03:59.220000


In [346]:
# Salvando e lendo em disco a lista de autores com abstracts
creator_vectors.to_json("Elsevier_abstract - Creators.json", date_format = 'iso')
creator_vectors = pd.read_json("Elsevier_abstract - Creators.json")

In [347]:
creator_vectors.head()

,Abstracts,Creator,Doc2Vec
0,"[complexes, of, and, aminopyridine, with, the,...","gerrard, d.l.","[0.1216731519, -1.1021842957, 4.7513136864, 4...."
1,"[complexes, of, and, aminopyridine, with, the,...","allan, j.r.","[1.0570453405, 0.12217671420000001, 5.93484735..."
10,"[small, angle, neutron, scattering, has, been,...","peiffer, d.g.","[0.6661279202, -7.5089697838, 2.51254320139999..."
100,"[dijet, angular, distributions, from, the, fir...","liang, z.","[-1.2167836428, -6.4154033661, -7.2966299057, ..."
101,"[the, chemisorption, and, thermal, decompositi...","gland, j.l.","[3.8563585281, -5.672191143, 4.7051215172, 8.6..."


É necessário reduzir a dimensionalidade dos vetores de 100 para 2 para conseguir gerar uma visualização. Será utilizado o algoritmo tSNE.

In [348]:
# Reduzindo de 100 dimensões para 2 usando tSNE para permitir a plotagem
tsne = TSNE(n_components=2, random_state=0, perplexity=12,early_exaggeration=10  )
embed_tsne = tsne.fit_transform(creator_vectors["Doc2Vec"].tolist())
embed_tsne = pd.DataFrame(data=embed_tsne)

In [3]:
# Plotando o grafico
scater_plot(embed_tsne, creator_vectors["Creator"], 'CornflowerBlue')

### Unindo os diversos vetores

Nesta seção será criado um dataframe com os três tipos de vetores: artigo, keyword e autor. Dessa forma, essas três entidades poderão ser analisadas no mesmo espaço vetorial

In [350]:
#Criando o dataframe multi_vector que irá receber os três tipos de vetores
multi_vector = pd.DataFrame(columns=['Type_vec', 'Description', 'Doc2Vec'])

In [351]:
#Adicionando os vetores dos artigos no multi_vector
multi_vector_article = pd.DataFrame(columns=['Type_vec', 'Description', 'Doc2Vec', 'Date', 'Affiliate'])
multi_vector_article [['Description', 
                       'Doc2Vec', 
                       'Date', 
                       'Affiliate']] = abstract_OG[['Title', 
                                                    'Doc2Vec', 
                                                    'Date', 
                                                    'Affiliate']]
#multi_vector_article ['Doc2Vec']= abstract_OG['Doc2Vec']
multi_vector_article['Type_vec'] = 'Article'
multi_vector_article = multi_vector_article#.head(1000)
multi_vector = pd.concat([multi_vector, multi_vector_article], ignore_index=True)

In [352]:
#Adicionando os vetores das keywords no multi_vector
multi_vector_keyword = pd.DataFrame(columns=['Type_vec', 'Description', 'Doc2Vec'])
multi_vector_keyword [['Description', 'Doc2Vec']]= keyword_vectors[['Keywords', 'Doc2Vec']]
multi_vector_keyword['Type_vec'] = 'Keywords'
multi_vector = pd.concat([multi_vector, multi_vector_keyword], ignore_index=True)

In [353]:
#Adicionando os vetores dos autores no multi_vector
multi_vector_creator = pd.DataFrame(columns=['Type_vec', 'Description', 'Doc2Vec'])
multi_vector_creator [['Description', 'Doc2Vec']]= creator_vectors[['Creator', 'Doc2Vec']]
multi_vector_creator['Type_vec'] = 'Creator'
multi_vector = pd.concat([multi_vector, multi_vector_creator], ignore_index=True)

In [354]:
# Reduzindo de 100 dimensões para 2 usando tSNE para permitir a plotagem
momentoInicial = datetime.datetime.now()   # Inicia um contador do tempo

tsne = TSNE(n_components=2, random_state=0, perplexity=12,early_exaggeration=10  )
embed_tsne = tsne.fit_transform(multi_vector["Doc2Vec"].tolist())
embed_tsne = pd.DataFrame(data=embed_tsne)

momentoFinal = datetime.datetime.now() #Encerrando o contador do tempo
print("Tempo total decorrido: ", momentoFinal - momentoInicial)        

Tempo total decorrido:  0:07:03.169000


In [355]:
# Acrescentando as duas dimensões do tSNE no dataframe
multi_vector["TSNE_1"] = embed_tsne[0]
multi_vector["TSNE_2"] = embed_tsne[1]

In [356]:
#Gravando o dataframe
multi_vector.to_csv("Elsevier_abstract - Multivector.csv")

In [357]:
# Realizando uma consulta de acordo com a coluna "Description" 
# e retornando os N vetores mais próximos
query = "oil"   #Termo consultado
n_answer = 20     #Número de respostas

# Indentificando o vetor da query
query_vector = multi_vector[multi_vector["Description"] == query]["Doc2Vec"].values[0]

#Criando um Dataframe para a query
multi_vector_query = multi_vector 

#Acrescentando uma coluna com as distâncias e selecionando o N mais próximos
multi_vector_query["DisQuery"] = (multi_vector["Doc2Vec"].
                                  apply(lambda x: distance.
                                        cosine(x , query_vector)))
multi_vector_query = (multi_vector_query.nsmallest(n_answer, "DisQuery")
                      .sort_values("DisQuery"))
multi_vector_query

,Affiliate,Date,Description,Doc2Vec,Type_vec,TSNE_1,TSNE_2,DisQuery
15074,NaN,NaT,oil,"[-0.4282711744, 2.7900283337, -2.6506662369000...",Keywords,23.724588,-54.072781,0.000000
6261,Petrobras,2010-01-30,Advanced performance evaluation of a reverse o...,"[0.1258293688, 6.7627344131000005, -6.18596124...",Article,-0.898239,-9.343952,0.048916
10610,Shell,2014-04-30,Economics of steam-assisted gravity drainage f...,"[-0.6078539491, 3.3439757824, -4.8164253235, -...",Article,-19.555229,-19.973049,0.049188
7743,Statoil,1999-04-30,Chemical and Ecotoxicological Characterisation...,"[-3.4217646122, 3.2996394634, -6.9708566666, -...",Article,-31.787167,26.226465,0.058726
7538,Eni,2008-02-15,Structure of waxy crude oil emulsion gels,"[-0.1932552606, 3.2740831375, -1.7274987698, -...",Article,24.381937,-54.424145,0.062128
3838,Exxon,2000-06-30,Characterizing PM2.5 emission profiles for sta...,"[-2.3404331207, 7.1500716209, -6.321516037, 0....",Article,-0.975951,-9.051226,0.064043
14934,NaN,NaT,classification,"[-3.6147005558, 4.3121452332, -4.9571299553, -...",Keywords,-31.555269,26.176226,0.069877
15285,NaN,NaT,produced water,"[-1.1359440088000001, 3.6240203381000002, -4.9...",Keywords,-34.611584,27.184467,0.072928
10502,Shell,2012-06-30,Well-to-wake energy and greenhouse gas analysi...,"[-1.0872951746, 4.3649663925, -5.7739739418, 2...",Article,-0.774618,-9.366707,0.072934
14497,NaN,NaT,petroleum,"[-0.2437291741, 3.0880243778, -1.3117181063, -...",Keywords,23.377129,-54.102676,0.080275


In [4]:
# Plotando o grafico
scater_plot([multi_vector_query["TSNE_1"],multi_vector_query["TSNE_2"]], 
            multi_vector_query["Description"], 
            'CornflowerBlue')